# Prerequisites
본 `ipynb` 은 `Python=3.12` 에서 작성하였습니다. Package dependency 를 해결하기 위해 아래 cell 을 실행해주세요.

## Install Python packages

In [ ]:
%pip -q install -U agent-framework

## Load environment variables from a .env file
secret 노출을 피하고 notebook 들간의 일관된 환경변수를 설정하기 위해 `dotenv` 을 이용한다.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

AZURE_MS_FOUNDRY_PROJECT_ENDPOINT = os.getenv("AZURE_MS_FOUNDRY_PROJECT_ENDPOINT")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_CHAT_DEPLOYMENT = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
AZURE_OPENAI_EMBEDDING_DEPLOYMENT = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

# Microsoft Agent Framework
다른 agent framework 과 비슷한 기능을 제공한다. gpt 모델의 client 를 만들고 agent 를 생성해보자.

In [ ]:
from agent_framework.azure import AzureAIClient
from azure.identity.aio import DefaultAzureCredential

agent = AzureAIClient(
    project_endpoint=AZURE_MS_FOUNDRY_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(),
    model_deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT,
).as_agent(name="buddy-agent", instructions="너는 내 친구야.")

result = await agent.run("안녕 내 이름은 이진호야. 너는 누구니 ?")
print(result.text)
print("--------------------------------")
result = await agent.run("내 이름은 뭐지 ?")
print(result.text)


대화 이력을 기억하기 위해선 thread 를 사용한다.

In [ ]:
t1 = agent.get_new_thread()

result = await agent.run("안녕 내 이름은 이진호야. 너는 누구니 ?", thread=t1)
print(result.text)
print("--------------------------------")
result = await agent.run("내 이름은 뭐지 ?", thread=t1)
print(result.text)


Function calling 도 지원한다. Agent 이기 때문에 tool_calling 에 대한 message 도 알아서 처리해준다.

In [ ]:
from typing import Annotated
from pydantic import Field
from agent_framework import tool

@tool(name="weather_tool", description="Retrieves weather information for any location")
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    return f"The weather in {location} is cloudy with a high of 15°C."

agent = AzureAIClient(
    project_endpoint=AZURE_MS_FOUNDRY_PROJECT_ENDPOINT,
    credential=DefaultAzureCredential(),
    model_deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT,
).as_agent(
    name="weather-agent", instructions="너는 날씨 정보를 제공하는 도우미야.", tools=get_weather,
)

result = await agent.run("서울의 날씨는 어때 ?")
print(result.text)

## Human-in-the-Loop
위 basic 한 내용은 그닥 어렵지 않다. 다소 advanced 한 기능을 한 번 넣어보면, HITL 이다.

In [ ]:
from typing import Annotated
from agent_framework import ChatAgent, ChatMessage, Role, tool
from agent_framework.azure import AzureOpenAIResponsesClient

@tool
def get_weather(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get the current weather for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C."

@tool(approval_mode="always_require")
def get_weather_detail(location: Annotated[str, "The city and state, e.g. San Francisco, CA"]) -> str:
    """Get detailed weather information for a given location."""
    return f"The weather in {location} is cloudy with a high of 15°C, humidity 88%."


agent = ChatAgent(
    chat_client=AzureOpenAIResponsesClient(
        endpoint=AZURE_OPENAI_ENDPOINT,
        api_key=AZURE_OPENAI_API_KEY,
        api_version=AZURE_OPENAI_API_VERSION,
        deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT,
    ),
    name="WeatherAgent",
    instructions="You are a helpful weather assistant.",
    tools=[get_weather, get_weather_detail],
)

thread = agent.get_new_thread()
result = await agent.run("뉴욕의 상세한 날씨는 어때?", thread=thread)
if result.user_input_requests:
    for user_input_needed in result.user_input_requests:
        print(f"Function: {user_input_needed.function_call.name}")
        print(f"Arguments: {user_input_needed.function_call.arguments}")
        approval = input(f"Do you approve the function call? (y/n)")
        approval = True if approval == "y" else False
        approval_message = ChatMessage(
            role=Role.USER, 
            contents=[user_input_needed.to_function_approval_response(approval)]
        )

        final_result = await agent.run(approval_message, thread=thread)
        print(final_result.text)

## Agent Observability
Microsoft Foundry 의 기능을 이용하면 remote 에서 동작하고 있는 agent 의 visibility 를 로깅할 수 있다. 아래를 살펴보자.

In [ ]:
import asyncio
import os
from random import randint
from typing import Annotated

from agent_framework import ChatAgent
from agent_framework import tool
from agent_framework.observability import create_resource, enable_instrumentation, get_tracer
from agent_framework.azure import AzureOpenAIResponsesClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry.trace import SpanKind
from opentelemetry.trace.span import format_trace_id
from pydantic import Field

@tool(approval_mode="never_require")
async def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    await asyncio.sleep(randint(0, 10) / 10.0)  # Simulate a network call
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

project_client = AIProjectClient(
    endpoint=AZURE_MS_FOUNDRY_PROJECT_ENDPOINT, credential=DefaultAzureCredential(),
)

# Microsoft Foundry 에서 제공하는 Application Insights 를 사용하기 위해 필요한 설정
conn_string = await project_client.telemetry.get_application_insights_connection_string()
# Application Insights 를 사용하기 위해 필요한 설정
configure_azure_monitor(
    connection_string=conn_string,
    enable_live_metrics=True,
    resource=create_resource(),
    enable_performance_counters=False,
)
# 민감한 데이터 수집 설정 (프로덕션 환경에서는 비활성화 필요)
enable_instrumentation(enable_sensitive_data=True)

with get_tracer().start_as_current_span("Weather Agent Chat", kind=SpanKind.CLIENT) as current_span:
    print(f"Trace ID: {format_trace_id(current_span.get_span_context().trace_id)}")
    agent = ChatAgent(
        chat_client=AzureOpenAIResponsesClient(
            endpoint=AZURE_OPENAI_ENDPOINT,
            api_key=AZURE_OPENAI_API_KEY,
            api_version=AZURE_OPENAI_API_VERSION,
            deployment_name=AZURE_OPENAI_CHAT_DEPLOYMENT,
        ),
        tools=get_weather,
        name="WeatherAgent",
        instructions="You are a weather assistant.",
        id="weather-agent",
    )

    thread = agent.get_new_thread()
    for question in ["삿포로 날씨는 어때 ?", "상하이 날씨와 비교해서 어디가 더 좋아?", "왜 하늘이 파래?"]:
        print(f"\nUser: {question}")
        print(f"{agent.name}: ", end="")
        async for update in agent.run_stream(
            question,
            thread=thread,
        ):
            if update.text:
                print(update.text, end="")